In [1]:
###############
## Libraries ##
###############

import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np


In [2]:
##############################################
## Load, partition, and resize MNIST Digits ##
##############################################
def loadData():
    all_data = np.load("/scratch/gpfs/eysu/src_data/mnist.npz")

    x_test = all_data['x_test']
    x_train = all_data['x_train']
    y_train = all_data['y_train']
    y_test = all_data['y_test']

    labels = ["0",  # index 0
              "1",  # index 1
              "2",  # index 2 
              "3",  # index 3 
              "4",  # index 4
              "5",  # index 5
              "6",  # index 6 
              "7",  # index 7 
              "8",  # index 8 
              "9"]  # index 9

    # save train labels
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255


    # y_train_labels = y_train
    # y_test_labels = y_test

    # Further break training data into train / validation sets (# put 5000 into validation set and keep remaining 55,000 for train)
    (x_train, x_valid) = x_train[5000:], x_train[:5000] 
    (y_train, y_valid) = y_train[5000:], np.array(y_train[:5000]).squeeze()

    # Reshape input data from (28, 28) to (28, 28, 1)
    w, h = 28, 28
    x_train = x_train.reshape(x_train.shape[0], w, h, 1)
    x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
    x_test = x_test.reshape(x_test.shape[0], w, h, 1)
    
    return x_train, x_valid, x_test, y_train, y_valid, y_test

In [3]:
############################################################
## Look at softmax output matrices for all sets of images ##
############################################################

# define constants and params
MAX_ITER = 25
labels = ["0",  # index 0
              "1",  # index 1
              "2",  # index 2 
              "3",  # index 3 
              "4",  # index 4
              "5",  # index 5
              "6",  # index 6 
              "7",  # index 7 
              "8",  # index 8 
              "9"]  # index 9

In [5]:
#######################################################
## Look at softmax output matrices for random images ##
#######################################################
# Number of learning iterations
MAX_ITER = 25
save_path = "/scratch/gpfs/eysu/mock_supervised_weights/"

y_hat_train_arr = np.zeros([y_train.shape[0], len(labels), MAX_ITER])
for i in range(MAX_ITER):
    if i == 0:
        y_hat_train_name = 'y_hat_train_seed'
       
    else:
        y_hat_train_name = 'y_hat_train_' + 'iter' + str(i)
        
    # Load test set softmax outputs 
    yhtr = np.load(save_path + y_hat_train_name + '.npy')

    # The first time through, use binary weight vectors to save seed array
    # Recall that these initial labels were randomized and do not correlate to 
    # the image's given class in the dataset
    
    if i == 0:
        true_class_tr = np.nonzero(yhtr)[1]   
    y_hat_train_arr[:, :, i] = yhtr

print(y_hat_train_arr.shape)
# (55000, 10, 25)

(55000, 10, 25)


In [6]:
print(true_class_tr)

[9 0 4 ... 6 6 3]


In [14]:
final_pred = y_hat_train_arr[:, :, 24]
print(final_pred.shape)
print(final_pred)
print(final_pred[0])

# ALL OUTPUTS THE SAME. NO INFO

(55000, 10)
[[0.09852035 0.10864645 0.10129254 ... 0.10201482 0.09894045 0.09865519]
 [0.09852035 0.10864645 0.10129254 ... 0.10201482 0.09894045 0.09865519]
 [0.09852035 0.10864645 0.10129254 ... 0.10201482 0.09894045 0.09865519]
 ...
 [0.09852035 0.10864645 0.10129254 ... 0.10201482 0.09894045 0.09865519]
 [0.09852035 0.10864645 0.10129254 ... 0.10201482 0.09894045 0.09865519]
 [0.09852035 0.10864645 0.10129254 ... 0.10201482 0.09894045 0.09865519]]
[0.09852035 0.10864645 0.10129254 0.10108736 0.09855531 0.09285692
 0.0994307  0.10201482 0.09894045 0.09865519]


In [42]:
# For CASE 2: 
avg_sim = 0
for END_IDX in range(1, 11):
    print(str(END_IDX) + ":")
    x_train, x_valid, x_test, y_train, y_valid, y_test = loadData()
    # create empty array to store softmax outputs
    y_hat_train_arr = np.zeros([y_train.shape[0], 10, MAX_ITER])

    for case in ["LR_adjusted/"]:
        save_path = "/scratch/gpfs/eysu/low_shot_weights/" + str(END_IDX) + "/" + case

        edit_name = False
        if case == "LR_adjusted/":
            edit_name = True

        # store the softmax vector from every iteration of training into y_hat_train_arr
        for i in range(MAX_ITER):
            if i == 0:
                if edit_name: y_hat_train_name = 'LR_adjustedy_hat_train_seed'
                else: y_hat_train_name = 'y_hat_train_seed'

            else:
                if edit_name: y_hat_train_name = 'LR_adjustedy_hat_train_' + 'iter' + str(i)
                else: y_hat_train_name = 'y_hat_train_' + 'iter' + str(i)

            # Load test set softmax outputs 
            yhtr = np.load(save_path + y_hat_train_name + '.npy')

            if i == 0:
                true_class_tr = np.nonzero(yhtr)[1]   
            y_hat_train_arr[:, :, i] = yhtr
            
            final_max_predictions = np.argmax(y_hat_train_arr[:,:, 1], axis=1)
            
    # final max predictions still only class 1
#     print(final_max_predictions.shape)
#     print(true_class_tr.shape)
    
    frac_correct = np.count_nonzero(np.where(final_max_predictions - true_class_tr == 0))/55000
    
    print(frac_correct)
    
#     cos_sim = np.dot(true_class_tr, final_max_predictions)/(np.linalg.norm(true_class_tr)*np.linalg.norm(final_max_predictions))
    
    avg_sim += frac_correct
#     print(cos_sim)
    final_pred = y_hat_train_arr[:, :, 24]

avg_sim = avg_sim/10
print(avg_sim)

1:
0.11234545454545454
2:
0.11232727272727273
3:
0.11556363636363637
4:
0.11234545454545454
5:
0.11270909090909091
6:
0.11245454545454546
7:
0.11252727272727273
8:
0.11234545454545454
9:
0.11230909090909091
10:
0.11238181818181818
0.11273090909090908


In [45]:
# For CASE 3: 
avg_sim = 0
for END_IDX in range(1, 11):
    print(str(END_IDX) + ":")
    x_train, x_valid, x_test, y_train, y_valid, y_test = loadData()
    # create empty array to store softmax outputs
    y_hat_train_arr = np.zeros([y_train.shape[0], 10, MAX_ITER])

    for case in ["shuffle_some/"]:
        save_path = "/scratch/gpfs/eysu/low_shot_weights/" + str(END_IDX) + "/" + case

        edit_name = False
        if case == "LR_adjusted/":
            edit_name = True

        # store the softmax vector from every iteration of training into y_hat_train_arr
        for i in range(MAX_ITER):
            if i == 0:
                if edit_name: y_hat_train_name = 'LR_adjustedy_hat_train_seed'
                else: y_hat_train_name = 'y_hat_train_seed'

            else:
                if edit_name: y_hat_train_name = 'LR_adjustedy_hat_train_' + 'iter' + str(i)
                else: y_hat_train_name = 'y_hat_train_' + 'iter' + str(i)

            # Load test set softmax outputs 
            yhtr = np.load(save_path + y_hat_train_name + '.npy')

            if i == 0:
                true_class_tr = np.nonzero(yhtr)[1]   
            y_hat_train_arr[:, :, i] = yhtr
            
            final_max_predictions = np.argmax(y_hat_train_arr[:,:, 14], axis=1)
            
    # final max predictions still only class 1
#     print(final_max_predictions.shape)
#     print(true_class_tr.shape)
    
    frac_correct = np.count_nonzero(np.where(final_max_predictions - true_class_tr == 0))/55000
    
    print(frac_correct)
    
#     cos_sim = np.dot(true_class_tr, final_max_predictions)/(np.linalg.norm(true_class_tr)*np.linalg.norm(final_max_predictions))
    
    avg_sim += frac_correct
#     print(cos_sim)
    final_pred = y_hat_train_arr[:, :, 24]
#     print(final_pred[0])

avg_sim = avg_sim/10
print(avg_sim)

1:
0.18725454545454545
2:
0.30185454545454543
3:
0.4073818181818182
4:
0.5077818181818182
5:
0.5991818181818181
6:
0.6921818181818182
7:
0.7899272727272727
8:
0.8936363636363637
9:
0.9904363636363637
10:
0.995890909090909
0.6365527272727273


In [49]:
# For CASE 4: 
avg_sim = 0
for END_IDX in range(1, 11):
    print(str(END_IDX) + ":")
    x_train, x_valid, x_test, y_train, y_valid, y_test = loadData()
    # create empty array to store softmax outputs
    y_hat_train_arr = np.zeros([y_train.shape[0], 10, MAX_ITER])

    for case in ["shuffle_none/"]:
        save_path = "/scratch/gpfs/eysu/low_shot_weights/" + str(END_IDX) + "/" + case

        edit_name = False
        if case == "LR_adjusted/":
            edit_name = True

        # store the softmax vector from every iteration of training into y_hat_train_arr
        for i in range(MAX_ITER):
            if i == 0:
                if edit_name: y_hat_train_name = 'LR_adjustedy_hat_train_seed'
                else: y_hat_train_name = 'y_hat_train_seed'

            else:
                if edit_name: y_hat_train_name = 'LR_adjustedy_hat_train_' + 'iter' + str(i)
                else: y_hat_train_name = 'y_hat_train_' + 'iter' + str(i)

            # Load test set softmax outputs 
            yhtr = np.load(save_path + y_hat_train_name + '.npy')

            if i == 0:
                true_class_tr = np.nonzero(yhtr)[1]   
            y_hat_train_arr[:, :, i] = yhtr
            
            final_max_predictions = np.argmax(y_hat_train_arr[:,:, 14], axis=1)
            
    # final max predictions still only class 1
#     print(final_max_predictions.shape)
#     print(true_class_tr.shape)
    
    frac_correct = np.count_nonzero(np.where(final_max_predictions - true_class_tr == 0))/55000
    
    print(frac_correct)
    
#     cos_sim = np.dot(true_class_tr, final_max_predictions)/(np.linalg.norm(true_class_tr)*np.linalg.norm(final_max_predictions))
    
    avg_sim += frac_correct
#     print(cos_sim)
    final_pred = y_hat_train_arr[:, :, 24]
#     print(final_pred[0])

avg_sim = avg_sim/10
print(avg_sim)

1:
0.9169818181818182
2:
0.9598545454545454
3:
0.9675818181818182
4:
0.9721818181818181
5:
0.9730363636363636
6:
0.9794545454545455
7:
0.9802909090909091
8:
0.9858363636363636
9:
0.9906
10:
0.9959090909090909
0.9721727272727273


In [38]:
# What about for unitialized model? 

MAX_ITER = 25
save_path = "/scratch/gpfs/eysu/mock_supervised_weights/"
    
true_class_tr = np.load("/scratch/gpfs/eysu/Serial-Reproductions-CNN-Research/weights_concise/weights_digits_1_trunc/y_hat_train_seed_DigitMNIST_TRAINED_.npy")
true_class_tr = np.nonzero(true_class_tr)[1]
final_pred = np.load("/scratch/gpfs/eysu/Serial-Reproductions-CNN-Research/weights_concise/weights_digits_1_trunc/y_hat_train_iter24_DigitMNIST_TRAINED_.npy")


final_max_predictions = np.argmax(final_pred, axis=1)

# final max predictions still only class 1
print(final_max_predictions.shape)
print(true_class_tr.shape)

frac_correct = np.count_nonzero(np.where(final_max_predictions - true_class_tr == 0))/55000

print(frac_correct)

    

(55000,)
(55000,)
0.9799272727272728
